In [1]:
import import_ipynb
from mergeData import HSE, HallUnits, SEUnits
from astropy import units as u
from astropy import constants as const
from units import UnitDataFrame

importing Jupyter notebook from mergeData.ipynb
importing Jupyter notebook from hall_data.ipynb
importing Jupyter notebook from rho_c_data.ipynb


In [2]:
allUnits = {**HallUnits, **SEUnits}

udf = UnitDataFrame(
    HSE.copy(),
    units = allUnits,
)

In [3]:
udf['rho_corr'] = (udf.t_AZO*udf.rho_sh).values.to(u.Ohm*u.m)
udf['n_corr']   = (udf.n_sh/udf.t_AZO).values.to(u.m**-3)
udf['R_H_corr'] = (udf.t_AZO*udf.R_H_sh).values.to(u.m**3/u.C)
#udf['omega_p'] = udf.omega_p.values.to(u.Hz, equivalencies = u.spectral()).to(u.rad/u.s, equivalencies = [(u.cycle/u.s, u.Hz)])

# convert to radians/s
udf['omega_p_rps'] = (udf.omega_p.values
    .to(u.Hz, equivalencies = u.spectral())
    .to(u.rad/u.s, equivalencies = [(u.cycle/u.s, u.Hz)])
    .to(1/u.s, equivalencies = u.dimensionless_angles())
    )

udf['omega_t_rps'] = (udf.omega_t.values
    .to(u.Hz, equivalencies = u.spectral())
    .to(u.rad/u.s, equivalencies = [(u.cycle/u.s, u.Hz)])
    .to(1/u.s, equivalencies = u.dimensionless_angles())
    )

udf['mu_opt'] = (
    (udf.omega_p_rps **2/udf.omega_t_rps)
    .values
    *const.eps0/(const.e.si*udf.n_corr.values.to(u.m**-3))
    ).to(u.cm**2/(u.V*u.s))

udf['m_eff'] = (
    const.e.si
    /(udf.omega_t_rps.values*udf.mu_opt.values)
    ).to(const.m_e)

In [4]:
udf.get_units()

{'MSE': None,
 'roughness': Unit("nm"),
 't_AlOx': Unit("nm"),
 't_AZO': Unit("nm"),
 'AZO_Einf': Unit("eV"),
 'TL_Amp': Unit("eV"),
 'TL_Br': Unit("eV"),
 'TL_E0': Unit("eV"),
 'TL_Eg': Unit("eV"),
 'omega_p': Unit("eV"),
 'omega_t': Unit("eV"),
 't_SiOx': Unit("nm"),
 't_int': Unit("nm"),
 'offset_angle': None,
 'doping': None,
 'capping': None,
 'anneal': None,
 'mu_H': Unit("m2 V / s"),
 'Carrier type': None,
 'n': Unit("1 / m3"),
 'n_sh': Unit("1 / m2"),
 'R_H': Unit("m3 / C"),
 'R_H_sh': Unit("m2 / C"),
 'rho': Unit("m Ohm"),
 'rho_sh': Unit("Ohm"),
 'V_H': Unit("V"),
 'd_AZO_Hall': Unit("nm"),
 'rho_corr': Unit("m Ohm"),
 'n_corr': Unit("1 / m3"),
 'R_H_corr': Unit("m3 / C"),
 'omega_p_rps': Unit("1 / s"),
 'omega_t_rps': Unit("1 / s"),
 'mu_opt': Unit("cm2 / (s V)"),
 'm_eff': Unit("9.10938e-31 kg")}

In [ ]:
# exports
df_export = udf
units_export = allUnits